In [0]:
synapse_password = dbutils.secrets.get(scope="meu-scope-kv", key="synapse-password")

In [0]:
# Get Paths
path_gold_quote = "abfss://gold@datalakeetlproject.dfs.core.windows.net/fact_quote"
temp_dir = "abfss://staging@datalakeetlproject.dfs.core.windows.net/fact_quote/"


In [0]:
#Datalake connection
blobAccessKey = dbutils.secrets.get(scope = "myscope", key = "accesskey")
spark.conf.set("fs.azure.account.key.datalakeetlproject.dfs.core.windows.net", 
               blobAccessKey) 

In [0]:
# Load Dataframe
df_fact_quote = spark.read.format("delta").load(path_gold_quote)

In [0]:
df_fact_quote.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", "jdbc:sqlserver://sy-elt-project.sql.azuresynapse.net:1433;database=dpool_eltproject;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;") \
  .option("user", "sqladminuser") \
  .option("password", synapse_password) \
  .option("dbtable", "dbo.fact_quote") \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("tempDir", temp_dir) \
  .mode("overwrite") \
  .save()
